In [1]:
import pandas as pd
import sys
import matplotlib.pyplot as plt

sys.path.append('../src')

from d00_utils.db_utils import dbReadWriteViews, dbReadWriteClean, dbReadWriteRaw

In [2]:
io_views = dbReadWriteViews()

all_inst_df = io_views.get_table('instances_unique_master_list')
label_inst_df = io_views.get_table('instances_with_labels')
confl_inst_df = io_views.get_table('instances_with_conflicts')

In [3]:
print(all_inst_df.shape)
all_inst_df.head()

(895167, 4)


,sopinstanceuid,instancefilename,instanceidk,studyidk
0,1.2.840.113543.6.6.4.6.62554526370968849932129...,6RYXZNZW,57157,2882
1,1.2.840.113543.6.6.4.6.64175920513291079284612...,6RYXZO03,57158,2882
2,1.2.840.113543.6.6.4.6.63515355813678132433121...,6RYXZO0D,57159,2882
3,1.2.840.113543.6.6.4.6.63385835434171778347132...,6RYXZO0K,57160,2882
4,1.2.840.113543.6.6.4.6.62250778798156715321013...,6RYXZO0Y,57161,2882


In [4]:
print(label_inst_df.shape)
label_inst_df.head()

(53799, 3)


,instanceidk,view,studyidk
0,57162,plax,2882
1,57164,a4c,2882
2,57571,plax,2915
3,58204,plax,2966
4,58713,plax,3009


In [5]:
print(confl_inst_df.shape)
confl_inst_df.head()

(13198, 2)


,instanceidk,studyidk
0,58207,2966
1,110441,6363
2,117616,6852
3,119931,6996
4,124996,7331


In [6]:
# for all studyidk's in label_inst_df, get all instanceidk's from all_inst_df
# delete instanceidk's that exist in label_inst_df
    # should decrease by 53965
# delete instanceidk's that exist in confl_inst_df

In [7]:
# get studies for which labels exist
studies_w_labels = list(set(label_inst_df['studyidk'].tolist()))
print('Number of unique studies for which we have labels')
print(len(label_inst_df['studyidk'].unique().tolist()))
print(len(studies_w_labels))

Number of unique studies for which we have labels
22073
22073


In [8]:
# get instances w labels - to be removed from all instances
inst_w_labels = list(set(label_inst_df['instanceidk'].tolist()))
print(len(inst_w_labels)) # should be same length as label_inst_df

# get instances w conflicts - to be removed from all instances
    # conflict i.e. instances s.t. is_multiview==True
inst_w_conflicts = list(set(confl_inst_df['instanceidk'].tolist()))
print(len(inst_w_conflicts)) # should be same as length confl_inst_df

53799
13198


In [9]:
# get all instanceidk's from studies which have measurements
poss_inst_df = all_inst_df[all_inst_df['studyidk'].isin(studies_w_labels)]
# studies_all = list(set(all_inst_df['studyidk'].tolist()))
print(poss_inst_df.shape)

# remove instances which have labels
poss_inst_df = poss_inst_df[~poss_inst_df['instanceidk'].isin(inst_w_labels)]
print(poss_inst_df.shape)

# remove instances which have conflicts
poss_inst_df = poss_inst_df[~poss_inst_df['instanceidk'].isin(inst_w_conflicts)]
print(poss_inst_df.shape)

(798839, 4)
(745040, 4)
(734176, 4)


In [10]:
print(poss_inst_df.shape)
poss_inst_df.head()

(734176, 4)


,sopinstanceuid,instancefilename,instanceidk,studyidk
0,1.2.840.113543.6.6.4.6.62554526370968849932129...,6RYXZNZW,57157,2882
1,1.2.840.113543.6.6.4.6.64175920513291079284612...,6RYXZO03,57158,2882
2,1.2.840.113543.6.6.4.6.63515355813678132433121...,6RYXZO0D,57159,2882
3,1.2.840.113543.6.6.4.6.63385835434171778347132...,6RYXZO0K,57160,2882
4,1.2.840.113543.6.6.4.6.62250778798156715321013...,6RYXZO0Y,57161,2882


In [11]:
# all_inst_df
lab_inst_df = io_views.get_table('instances_with_labels')
print(lab_inst_df.shape)
lab_inst_df.head()

(53799, 3)


,instanceidk,view,studyidk
0,57162,plax,2882
1,57164,a4c,2882
2,57571,plax,2915
3,58204,plax,2966
4,58713,plax,3009


In [12]:
inst_fair_game = list(set(all_inst_df['instanceidk'].tolist()))

lab_inst_fair_df = lab_inst_df[lab_inst_df['instanceidk'].isin(inst_fair_game)]
print(lab_inst_fair_df.shape)
lab_inst_fair_df.head()

(53799, 3)


,instanceidk,view,studyidk
0,57162,plax,2882
1,57164,a4c,2882
2,57571,plax,2915
3,58204,plax,2966
4,58713,plax,3009


In [13]:
io_clean = dbReadWriteClean()

meta_df = io_clean.get_table('meta_lite')

In [34]:
summ_df = io_clean.get_table('dm_spain_view_study_summary')
cols_to_drop = ['age','gender','patientweight','patientheight','findingcode','findingcodetext','conclusion','reasonforstudy', 'studylocation','modality']
machine_df = summ_df.drop(labels=cols_to_drop,axis=1)
machine_df.head()

# Table that contains machine type and bmi for every study
io_views.save_to_db(machine_df, 'machines_all_bmi')

In [31]:
# Waiting for Jesus: which machines do we want to keep?
# #machine_df['equipment'].value_counts()

machines_to_keep = ['ECOEPIQ2', 'EPIQ7-1', 'ECOIE33', 'AFFINITI_1', 'AFFINITI_2']
machine_new_df = machine_df[machine_df['equipment'].isin(machines_to_keep)]
print(machine_new_df.shape)
machine_new_df.head()

(14562, 3)


,studyidk,equipment,bmi
0,118640,EPIQ7-1,29.340040
1,115777,ECOEPIQ2,18.256319
2,49414,ECOIE33,24.447279
3,79460,AFFINITI_1,24.092971
4,118806,AFFINITI_2,22.589551


In [35]:
# Table that contains machine type and bmi for studies on new machines
# New machines: ECOEPIQ2, EPIQ7-1, ECOIE33, AFFINITI_1, AFFINITI_2
io_views.save_to_db(machine_new_df, 'machines_new_bmi')

In [36]:
# now need to make 'views.instances_with_labels_new_machines'
lab_inst_df = io_views.get_table('instances_with_labels')
print(lab_inst_df.shape)
lab_inst_df.head()

(53799, 3)


,instanceidk,view,studyidk
0,57162,plax,2882
1,57164,a4c,2882
2,57571,plax,2915
3,58204,plax,2966
4,58713,plax,3009


In [37]:

#inst_fair_game = list(set(all_inst_df['instanceidk'].tolist()))
studies_new_machines = list(set(machine_new_df['studyidk'].tolist()))

lab_inst_new_df = lab_inst_df[lab_inst_df['studyidk'].isin(studies_new_machines)]

print(lab_inst_new_df.shape)
lab_inst_new_df.head()

## PROBABLY WANT SOPINSTANCEUID and instancefilename on this table!

(32554, 3)


,instanceidk,view,studyidk
2359,696108,plax,39531
2360,696109,plax,39531
2361,696112,a4c,39531
2362,696140,plax,39533
2363,696157,plax,39533


In [43]:
32554.0 / 53799.0

0.6051041840926411

In [44]:
439232.0 / 734181.0

0.5982611917224772

In [55]:
oth_inst_df = io_views.get_table('instances_labeled_other')
oth_inst_df['view'] = 'other'
print(oth_inst_df.shape)
oth_inst_df.head()

(439226, 5)


,sopinstanceuid,instancefilename,instanceidk,studyidk,view
0,1.2.840.113663.1500.1.365900332.3.1.20120703.9...,8Z0BWVQI,696106,39531,other
1,1.2.840.113663.1500.1.365900332.3.2.20120703.9...,8Z0BWW6Y,696107,39531,other
2,1.2.840.113663.1500.1.365900332.3.5.20120703.9...,8Z0BWY3Y,696110,39531,other
3,1.2.840.113663.1500.1.365900332.3.6.20120703.9...,8Z0BWYQG,696111,39531,other
4,1.2.840.113663.1500.1.365900332.3.8.20120703.9...,8Z0BWZJ9,696113,39531,other


In [42]:
oth_inst_new_df = oth_inst_df[oth_inst_df['studyidk'].isin(studies_new_machines)]
print(oth_inst_new_df.shape)

(439232, 4)


In [56]:
master_df = io_views.get_table('instances_unique_master_list')
print(master_df.shape)
master_df.head()

(895167, 4)


,sopinstanceuid,instancefilename,instanceidk,studyidk
0,1.2.840.113543.6.6.4.6.62554526370968849932129...,6RYXZNZW,57157,2882
1,1.2.840.113543.6.6.4.6.64175920513291079284612...,6RYXZO03,57158,2882
2,1.2.840.113543.6.6.4.6.63515355813678132433121...,6RYXZO0D,57159,2882
3,1.2.840.113543.6.6.4.6.63385835434171778347132...,6RYXZO0K,57160,2882
4,1.2.840.113543.6.6.4.6.62250778798156715321013...,6RYXZO0Y,57161,2882


In [57]:
lab_inst_df = io_views.get_table('instances_with_labels')
print(lab_inst_df.shape)
lab_inst_df.head()

(32554, 3)


,instanceidk,view,studyidk
0,696108,plax,39531
1,696109,plax,39531
2,696112,a4c,39531
3,696140,plax,39533
4,696157,plax,39533


In [51]:
#studies_relevant = list(set(lab_inst_df['studyidk'].to_list()))
#master_relevant_df = master_df[master_df['studyidk'].isin(studies_relevant)]

print(master_relevant_df.shape)

(477465, 4)


In [58]:
# merge_df = master_relevant_df.merge(lab_inst_df, on='instanceidk')
merge_df = master_df.merge(lab_inst_df, on='instanceidk')

print(merge_df.shape)
merge_df.head()

(32554, 6)


,sopinstanceuid,instancefilename,instanceidk,studyidk_x,view,studyidk_y
0,1.2.840.113663.1500.1.365900332.3.3.20120703.9...,8Z0BWWL5,696108,39531,plax,39531
1,1.2.840.113663.1500.1.365900332.3.4.20120703.9...,8Z0BWX0M,696109,39531,plax,39531
2,1.2.840.113663.1500.1.365900332.3.7.20120703.9...,8Z0BWZGH,696112,39531,a4c,39531
3,1.2.840.113663.1500.1.365900332.3.1.20120703.9...,8Z0BXKCB,696140,39533,plax,39533
4,1.2.840.113663.1500.1.365900332.3.14.20120703....,8Z0BXYUW,696157,39533,plax,39533


In [59]:
merge_df = merge_df[merge_df['studyidk_x'] == merge_df['studyidk_y']]
merge_df['studyidk'] = merge_df['studyidk_x']
cols_to_drop = ['studyidk_x', 'studyidk_y']
merge_df.drop(labels=cols_to_drop, axis=1, inplace=True)
print(merge_df.shape)
merge_df.head()

(32554, 5)


,sopinstanceuid,instancefilename,instanceidk,view,studyidk
0,1.2.840.113663.1500.1.365900332.3.3.20120703.9...,8Z0BWWL5,696108,plax,39531
1,1.2.840.113663.1500.1.365900332.3.4.20120703.9...,8Z0BWX0M,696109,plax,39531
2,1.2.840.113663.1500.1.365900332.3.7.20120703.9...,8Z0BWZGH,696112,a4c,39531
3,1.2.840.113663.1500.1.365900332.3.1.20120703.9...,8Z0BXKCB,696140,plax,39533
4,1.2.840.113663.1500.1.365900332.3.14.20120703....,8Z0BXYUW,696157,plax,39533
